In [37]:
import sys
sys.path.append('../')

import time
import warnings
import pandas as pd
import xgboost as xgb
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from utils.features.feature_extraction import Feature_extraction

from utils.constant import FEATURES, LABELS, SYN_FLOOD_ATTACKS

In [38]:
start = time.time()
pcapfiles = [
    '../inputs/sniffed.pcap',
]
destination_directory = '../outputs/'
n_threads = 8

address = "./"

for i in range(len(pcapfiles)):
    lstart = time.time()
    pcap_file = pcapfiles[i]
    fe = Feature_extraction()
    df = fe.pcap_evaluation(pcap_file,destination_directory + pcap_file)
    print(f'done! ({pcap_file})(' + str(round(time.time()-lstart, 2))+ 's)')
    
end = time.time()
print(f'Elapsed Time = {(end-start)}s')
df

done! (../inputs/sniffed.pcap)(0.59s)
Elapsed Time = 0.589646577835083s


,ts,Protocol_name,Duration,Protocol Type,flow_duration,Header_Length,src_ip_bytes,fin_flag_number,syn_flag_number,rst_flag_number,...,Weight,Correlation,RARP,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time,source_ip
0,1.701051e+09,UDP,128,17,0.000000,99,1,0,0,0,...,1,0.0,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.110
1,1.701051e+09,UDP,128,17,0.000000,99,2,0,0,0,...,4,NaN,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.110
2,1.701051e+09,UDP,64,17,0.053003,306,1,0,0,0,...,9,1.0,0,0,0,0,0,5.300307e-02,0.053003,192.168.43.65
3,1.701051e+09,UDP,64,17,0.056315,294,2,0,0,0,...,16,1.0,0,0,0,0,0,5.631495e-02,0.056315,192.168.43.65
4,1.701051e+09,TCP,128,6,0.000000,66,3,0,1,0,...,25,1.0,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1.701051e+09,TCP,64,6,0.000000,54,414,0,1,0,...,196,NaN,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.221
874,1.701051e+09,TCP,64,6,0.000000,54,415,0,1,0,...,225,NaN,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.221
875,1.701051e+09,TCP,64,6,0.000000,54,416,0,1,0,...,256,NaN,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.221
876,1.701051e+09,TCP,64,6,0.000000,54,417,0,1,0,...,289,NaN,0,0,0,0,0,1.701051e+09,0.000000,192.168.43.221


In [39]:
# Preprocessing Functions
scaler = StandardScaler()
scaler.fit(df[FEATURES])
df[FEATURES] = scaler.transform(df[FEATURES])

df[df['source_ip'] == '192.168.43.221'][FEATURES]

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
460,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,-1.476974,-0.728197,-0.725937,-0.616697,-0.938129,-1.161600
461,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,-1.303212,-0.728197,-0.725937,-0.616697,-0.938129,-1.136718
462,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,-1.129450,-0.728197,-0.725937,-0.616697,-0.938129,-1.095247
463,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,-0.955689,-0.728197,-0.725937,-0.616697,-0.938129,-1.037188
464,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,-0.781927,-0.728197,-0.725937,-0.616697,-0.938129,-0.962540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,0.781927,-0.728197,-0.725937,-0.616697,-0.938129,0.455764
874,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,0.955689,-0.728197,-0.725937,-0.616697,-0.938129,0.696295
875,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,1.129450,-0.728197,-0.725937,-0.616697,-0.938129,0.953415
876,-0.186258,-0.740585,-0.127294,-0.895409,-0.031879,-0.031879,0.0,-0.218478,0.957617,-0.095893,...,-0.634135,-0.726064,-0.410962,-0.229691,1.303212,-0.728197,-0.725937,-0.616697,-0.938129,1.227123


# Predict with our model

In [40]:
xgb_model = xgb.XGBClassifier()
xgb_model.load_model('../outputs/xgb.model.bin.json')

In [41]:
preds = xgb_model.predict(df[FEATURES])
import numpy as np
idx = np.where(preds == 1)[0]
idx_2 = np.where(df['source_ip']=='192.168.43.221')[0]
print(len(idx))
print(len(idx_2))

665
418
